# Cricket Player Rating System


Data cleaning & pre-processing, feature engineering and creation of an algorithm to rate cricket players and discover their overall ranking:

### Import Libraries:

In [ ]:
import pandas as pd
import numpy as np

## Data Cleaning and Pre-processing

First we load the dataset:

In [7]:
# Load the dataset
df = pd.read_csv(r'datasets\formatted_bbb_df.csv')

In [8]:
print('Number of rows:', df.shape[0])
print('Number of columns:', df.shape[1])

Number of rows: 1283363
Number of columns: 88


Next we can convert out date column to datetime:

In [9]:
display(df['date'].head(3))

# Convert our date column to datetime 
df['date'] = df['date'].str[:10]
df['date'] = pd.to_datetime(df['date'], format='mixed')

display(df['date'].head(3))

0    2006-01-09T00:00Z
1    2006-01-09T00:00Z
2    2006-01-09T00:00Z
Name: date, dtype: object

0   2006-01-09
1   2006-01-09
2   2006-01-09
Name: date, dtype: datetime64[ns]

We can create a function to create a subset of our dataset:

In [11]:
def filter_dataframe_by_date(df, lower_date=None, upper_date=None):
    
    if df['date'].dtype == 'object':
        # Convert dates to datetime
        df['date'] = df['date'].str[:10]
        df['date'] = pd.to_datetime(df['date'], format='mixed')

    # Convert lower_date and upper_date to pandas datetime if they are strings
    if isinstance(lower_date, str):
        lower_date = pd.to_datetime(lower_date)
    if isinstance(upper_date, str):
        upper_date = pd.to_datetime(upper_date)

    # Determine the earliest and latest dates in the dataframe
    min_date = df['date'].min()
    max_date = df['date'].max()

    # Assign the earliest or latest date if lower_date or upper_date is not provided
    lower_date = lower_date or min_date
    upper_date = upper_date or max_date

    # Filter the dataframe based on the date range
    filtered_dataframe = df[(df['date'] >= lower_date) & (df['date'] <= upper_date)]

    return filtered_dataframe

In [12]:
df = filter_dataframe_by_date(df, lower_date='2010-01-01', upper_date='2015-01-01')

print('Number of rows:', df.shape[0])
print('Number of columns:', df.shape[1])

Number of rows: 301622
Number of columns: 88


We can check the max value for the overs column to see if we are only dealing with T20 data:

In [13]:
# Show the max value in the overs column
print('Max number of overs:', df['overs'].max())

Max number of overs: 50.0


Since we clearly have data for other tournaments with higher over limits in this dataset, we can remove these:

In [14]:
rows_before = df.shape[0]

# Identify event IDs with overs greater than 20
event_ids_to_remove = df.loc[df['overs'] > 20, 'event_id'].unique()

# Remove rows with event IDs that have overs greater than 20
df = df.loc[~df['event_id'].isin(event_ids_to_remove)]

rows_after = df.shape[0]
print(rows_before-rows_after, 'rows removed')


40597 rows removed


Similarly, we remove entries that have an over_limit value of over 20:

In [15]:
rows_before = df.shape[0]

# Identify event IDs with overs greater than 20
event_ids_to_remove = df.loc[df['over_limit'] > 20, 'event_id'].unique()

# Remove rows with event IDs that have overs greater than 20
df = df.loc[~df['event_id'].isin(event_ids_to_remove)]

rows_after = df.shape[0]
print(rows_before-rows_after, 'rows removed')

0 rows removed


Since we will need player names for our final ratings, we can build functions that check that each id for a player has a unique name, and if not to replace the name with the most common name for that ID:

In [16]:
# Function to check number of names for each id
def check_duplicate_names_for_id(df, id, name):
    num_duplicate_names = (df.groupby(id)[name].nunique() > 1).sum()
    return num_duplicate_names

In [17]:
# Function to replace duplicate names in the DataFrame for a given id and name column
def replace_duplicate_names(df, id, name):
    
    num_duplicates = check_duplicate_names_for_id(df, id, name)
    
    if num_duplicates == 0:
        print('No duplicate name(s) for:', id)
    else:        
        print(f'{num_duplicates} duplicate names in {id}')
        
        # Get the ids with the most duplicate names, sorted in descending order
        duplicate_list = df.groupby(id)[name].nunique().sort_values(ascending=False).head(num_duplicates)
        
        # Iterate through the duplicate ids
        for duplicate_id in duplicate_list.index:
            # Get the unique names for the current duplicate id, sorted by frequency
            unique_names = df[df[id] == duplicate_id][name].value_counts().sort_values(ascending=False).index
            
            # Select the most common name as the top name
            top_name = unique_names[0]
            
            # Iterate through the remaining unique names
            for player_name in unique_names[1:]:
                # Replace each unique name with the top name in the dataframe
                df[name] = df[name].replace({player_name: top_name})
                
                print(f'{player_name} replaced with {top_name}')
    
    return df

We then define all the id and name pairs and run the above functions to ensure each player ID has a unique name:

In [18]:
# Create a dictionary of player id and name pairs
role_id_names = {'bowler_id': 'bowler_name',
            'batter_id': 'batsman_striker_name'}

In [19]:
# Iterate through the dictionary and replace duplicates
for id, name in role_id_names.items():
    replace_duplicate_names(df, id, name)
    print('\n')

No duplicate name(s) for: bowler_id


No duplicate name(s) for: batter_id




We can do the same for the team IDs and team names to ensure there are no duplicates here:

In [20]:
# Create a dictionary of team id and name pairs
team_id_names = {'bowler_team_id': 'bowler_team_name',
            'batsman_striker_team_id': 'batsman_striker_team_name'}

In [21]:
# Iterate through the dictionary and replace duplicates
for id, name in team_id_names.items():
    replace_duplicate_names(df, id, name)
    print('\n')

No duplicate name(s) for: bowler_team_id


No duplicate name(s) for: batsman_striker_team_id




We can see that every team ID has a unique team name. 

Now we will look into missing values for the bowler_id and batter_id columns:

In [22]:
# Print the count of missing 'bowler_id' values
print(df['bowler_id'].isna().sum(), 'missing bowler_id values')

134 missing bowler_id values


We can see that we have a notable number of missing values for bowler_id. We can have a look at some of these rows:

In [23]:
# Retrieve the first few rows where 'bowler_id' is NaN
df[df['bowler_id'].isna()].head()

,id,event_id,innings,overs,ball_no,match_ball_no,innings_runs,innings_wickets,innings_target,innings_remaining_runs,...,over_wickets,over_actual,over_unique,dismissal_dismissal,dismissal_bowled,dismissal_minutes,dismissal_bowler_id,dismissal_bowler_name,dismissal_batsman_id,dismissal_batsman_name
83148,110010,440100,1.0,10.1,1.0,61.0,66.0,6.0,0.0,NaN,...,0.0,10.1,10.01,False,False,0.0,NaN,NaN,25483.0,Simon Smith
83149,110020,440100,1.0,10.2,2.0,62.0,66.0,6.0,0.0,NaN,...,0.0,10.2,10.02,False,False,0.0,NaN,NaN,25475.0,Ross Lyons
83150,110030,440100,1.0,10.3,3.0,63.0,66.0,6.0,0.0,NaN,...,0.0,10.3,10.03,False,False,0.0,NaN,NaN,25475.0,Ross Lyons
83151,110040,440100,1.0,10.4,4.0,64.0,66.0,6.0,0.0,NaN,...,0.0,10.4,10.04,False,False,0.0,NaN,NaN,25475.0,Ross Lyons
83152,110050,440100,1.0,10.5,5.0,65.0,66.0,6.0,0.0,NaN,...,0.0,10.5,10.05,False,False,0.0,NaN,NaN,25475.0,Ross Lyons


Clearly there is some messy data here so we can remove all rows with this ID value

In [24]:
# Get the old number of rows in the dataframe 
rows_before = df.shape[0]

# Drop rows where 'id' matches 999999999999999
df = df.drop(df[df['id'] == 999999999999999].index)

# Get the new number of rows in the dataframe 
rows_after = df.shape[0]

# Show how many rows were removed
print(rows_before-rows_after, 'rows removed')

39 rows removed


We can no look into filling null values for bowler_id. 

We know that the same bowler will be bowling every ball for a certain over in an innings in a match, therefore we can forward fill all missing values for this over from values we have in this over:

In [25]:
# Print the old count of missing 'bowler_id' values
print(df['bowler_id'].isna().sum(), 'missing bowler_id values')

# Forward fill missing values based on 'event_id', 'innings', and the same value before the decimal point for 'overs'
df['bowler_id'] = df.groupby(['event_id', 'innings', df['overs'].apply(int)])['bowler_id'].fillna(method='ffill')

# Print the new count of missing 'bowler_id' values
print(df['bowler_id'].isna().sum(), 'missing bowler_id values')


95 missing bowler_id values
15 missing bowler_id values


This has allowed us to fill a significant number of missing values. We could fill the further missing values through external research, but giving that this is only a small percentage we will leave them.

We can create a function to impute missing names from their ID:

In [26]:
# Function to fill names from name ID
def fill_name_from_id(df, id, name):

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        if pd.isnull(row[name]):
            # Retrieve the bowler_id for the current row
            player_id = row[id]

            # Find matching names for the bowler_id
            matching_names = df.loc[df[id] == player_id, name]

            # If matching names exist, assign the first matching name to the current row's 'bowler_name'
            if not matching_names.empty:
                df.at[index, name] = matching_names.values[0]

    return df

In [27]:
# Print the count of missing 'bowler_name' values
print(df['bowler_name'].isna().sum(), 'missing bowler_name values')

# Use the function to fill missing names from ID
fill_name_from_id(df, 'bowler_id', 'bowler_name')

# Print the count of missing 'bowler_name' values after filling missing values
print(df['bowler_name'].isna().sum(), 'missing bowler_name values')


95 missing bowler_name values
15 missing bowler_name values


We can finally fill all remaining missing bowler_id and bowler_name values with an Unknown value:

In [28]:
# Fill missing values in 'bowler_id' column with 'Unknown'
df['bowler_id'] = df['bowler_id'].fillna('Unknown')

# Fill missing values in 'bowler_name' column with 'Unknown'
df['bowler_name'] = df['bowler_name'].fillna('Unknown')

# Print the count of missing 'bowler_name' values
print(df['bowler_name'].isna().sum(), 'missing bowler_name values')

0 missing bowler_name values


We can now look into filling missing batter_id values:

In [29]:
# Print the count of missing 'batter_id' values
print(df['batter_id'].isna().sum(), 'missing batter_id values')

9 missing batter_id values


Here, we know that for any match and innings, the batter should remain the same provided there are no wickets lost and the runs scored is of an even value. Therefore we can use this information to fill in any missing values for batter_id:

In [30]:
# Create a mask based on specific conditions
mask = (
    ((df['score_value'].shift(1) % 2 == 0) & (df['wickets_lost'].shift(1) == 0)) &  # If the previous score is even and no wickets were lost
    (df['event_id'].shift(1) == df['event_id']) &  # If the previous event_id matches the current event_id
    (df['innings'].shift(1) == df['innings'])  # If the previous innings value matches the current innings value
)

# Initialise variables for missing count
prev_missing_count = float('inf')
curr_missing_count = df['batter_id'].isna().sum()

# Iteratively fill missing values until the count stabilizes
while curr_missing_count != prev_missing_count:
    prev_missing_count = curr_missing_count

    # Fill missing values in the 'batter_id' column using the previous non-missing value
    df.loc[mask, 'batter_id'] = df['batter_id'].fillna(df['batter_id'].shift(1))

    # Update the current missing count
    curr_missing_count = df['batter_id'].isna().sum()

# Print the new count of missing 'batter_id' values
print(curr_missing_count, 'missing batter_id values')


8 missing batter_id values


Again, while we could fill in the further missing values through further research, we will keep these blank to save time. 

Similarly to before, we can use the filled ID values to fill in the batter name values

In [31]:
# Print the count of missing 'batsman_striker_name' values
print(df['batsman_striker_name'].isna().sum(), 'missing batsman_striker_name values')

# Use the function to fill missing names from ID
fill_name_from_id(df, 'batter_id', 'batsman_striker_name')

# Print the count of missing 'batsman_striker_name' values after filling missing values
print(df['batsman_striker_name'].isna().sum(), 'missing batsman_striker_name values')

9 missing batsman_striker_name values
8 missing batsman_striker_name values


Finally, again we can fill the missing values with 'Unknown'

In [32]:
# Fill missing values in 'batter_id' column with 'Unknown'
df['batter_id'] = df['batter_id'].fillna('Unknown')

# Fill missing values in 'batsman_striker_name' column with 'Unknown'
df['batsman_striker_name'] = df['batsman_striker_name'].fillna('Unknown')

# Print the count of missing 'batsman_striker_name' values
print(df['batsman_striker_name'].isna().sum(), 'missing batsman_striker_name values')

0 missing batsman_striker_name values


We will now create a feature that shows whether the current batter players for the home side. We can do this by checking whether the away score is 0 in each case:

In [33]:
# Set the initial value of the 'is_home_batter' column to 0 for all rows
df['is_home_batter'] = 0

# Update the 'is_home_batter' column to 1 for rows where the 'away_score' is '0'
df.loc[df['away_score'] == '0', 'is_home_batter'] = 1


For each event_id, each player should only have one value for is_home_batter, we can check that this is definitely the case:

In [34]:
# Group the dataframe by 'event_id' and 'batter_id', then count the number of unique values in 'is_home_batter' column
grouped = df.groupby(['event_id', 'batter_id'])['is_home_batter'].nunique()

# Filter the groups where the count of unique values is greater than 1
result = grouped.loc[lambda x: x > 1]

print(result)

event_id  batter_id
693019    21577.0      2
          230855.0     2
Name: is_home_batter, dtype: int64


While we would expect to have discrepancies for 'Unknown', there are two players in one match that have two values for is_home_batter so we can look further into this match:

In [35]:
# Filter the DataFrame based on specific conditions
filtered_df = df[(df['event_id'] == 693019) & (df['batter_id'] == 21577.0)]

# Select specific columns from the filtered DataFrame
selected_columns = filtered_df[['event_id', 'innings', 'batter_id', 'batsman_striker_name', 'batsman_striker_team_name']]

# Retrieve the last 3 rows of the selected columns
selected_columns.tail(3)

,event_id,innings,batter_id,batsman_striker_name,batsman_striker_team_name
333209,693019,1.0,21577.0,Peter Trego,Somerset
333210,693019,1.0,21577.0,Peter Trego,Somerset
333239,693019,2.0,21577.0,Peter Trego,Gloucestershire


In [36]:
# Filter the DataFrame based on specific conditions
filtered_df = df[(df['event_id'] == 693019) & (df['batter_id'] == 230855.0)]

# Select specific columns from the filtered DataFrame
selected_columns = filtered_df[['event_id', 'innings', 'batter_id', 'batsman_striker_name', 'batsman_striker_team_name']]

# Retrieve the last 3 rows of the selected columns
selected_columns.tail(3)

,event_id,innings,batter_id,batsman_striker_name,batsman_striker_team_name
333225,693019,1.0,230855.0,Craig Kieswetter,Somerset
333231,693019,1.0,230855.0,Craig Kieswetter,Somerset
333276,693019,2.0,230855.0,Craig Kieswetter,Gloucestershire


We can see above that there was clearly an issue collecting data for this particular match. Therefore, we can we define a function that replaces discrepancies such as these wiht 'Unknown':

In [37]:
# Function to replace discrepancies in specified columns with 'Unknown' values in the DataFrame.
def replace_discrepancies_with_unknown(df, discrepancy_column, column_to_replace1, column_to_replace2):

    # Group the DataFrame by 'event_id' and the first replacement column, and count the number of unique values in the discrepancy column
    discrepancy_df = df.groupby(['event_id', column_to_replace1])[discrepancy_column].nunique().loc[lambda x: x > 1].reset_index()[['event_id', column_to_replace1]]

    for i, row in discrepancy_df.iterrows():
        
        # Define event_id and batter_id
        event_id = row['event_id']
        id = row[column_to_replace1]

        if id == 'Unknown':
            continue

        else:

            # Filter the DataFrame based on the specified conditions
            filtered_df = df.loc[(df['event_id'] == event_id) & (df[column_to_replace1] == id)]

            # Calculate the most common value
            most_common_value = filtered_df[discrepancy_column].mode().values[0]

            # Replace any value except the most common value with 'Unknown Batter'
            filtered_df.loc[filtered_df[discrepancy_column] != most_common_value, column_to_replace1] = 'Unknown'
            filtered_df.loc[filtered_df[discrepancy_column] != most_common_value, column_to_replace2] = 'Unknown'

            # Update the original DataFrame with the modified values
            df.update(filtered_df)
    
    return df

In [38]:
# Run the above function on discrepancies with is_home_batter
df = replace_discrepancies_with_unknown(df, 'is_home_batter', 'batter_id', 'batsman_striker_name')

We can now check that Unknown is the only value remaining with multiple unique values for is_home_batter 

In [39]:
# Group the dataframe by 'event_id' and 'batter_id', then count the number of unique values in 'is_home_batter' column
grouped = df.groupby(['event_id', 'batter_id'])['is_home_batter'].nunique()

# Filter the groups where the count of unique values is greater than 1
result = grouped.loc[lambda x: x > 1]

print(result)

Series([], Name: is_home_batter, dtype: int64)


We can now do a similar process for is_home_bowler:

In [40]:
# Assign 0 to the 'is_home_bowler' column for all rows in the DataFrame
df['is_home_bowler'] = 0

# Set the value of 'is_home_bowler' to 1 for rows where the 'home_score' column is '0'
df.loc[df['home_score'] == '0', 'is_home_bowler'] = 1

In [41]:
# Group the dataframe by 'event_id' and 'bowler_id', then count the number of unique values in 'is_home_bowler' column
grouped = df.groupby(['event_id', 'bowler_id'])['is_home_bowler'].nunique()

# Filter the groups where the count of unique values is greater than 1
result = grouped.loc[lambda x: x > 1]

print(result)

event_id  bowler_id
693019    211748.0     2
          451782.0     2
Name: is_home_bowler, dtype: int64


We can see that this is again an issue with the collected data for the same match as before:

In [42]:
# Filter the DataFrame based on specific conditions
filtered_df = df[(df['event_id'] == 693019) & (df['bowler_id'] == 211748.0)]

# Select specific columns from the filtered DataFrame
selected_columns = filtered_df[['event_id', 'innings', 'bowler_id', 'bowler_name', 'bowler_team_name']]

# Retrieve the last three rows from the selected columns
selected_columns.tail(3)

,event_id,innings,bowler_id,bowler_name,bowler_team_name
333227,693019,1.0,211748.0,Benny Howell,Gloucestershire
333228,693019,1.0,211748.0,Benny Howell,Gloucestershire
333276,693019,2.0,211748.0,Benny Howell,Somerset


Therefore, we can use our defined function from before to replace the incorrect value with 'Unknown'

In [43]:
# Run the above function on discrepancies with is_home_bowler
df = replace_discrepancies_with_unknown(df, 'is_home_bowler', 'bowler_id', 'bowler_name')

We can again check that the discrepacnies have been removed:

In [44]:
# Group the dataframe by 'event_id' and 'bowler_id', then count the number of unique values in 'is_home_bowler' column
grouped = df.groupby(['event_id', 'bowler_id'])['is_home_bowler'].nunique()

# Filter the groups where the count of unique values is greater than 1
result = grouped.loc[lambda x: x > 1]

print(result)

Series([], Name: is_home_bowler, dtype: int64)


We can also look into events that only have one value for innings and remove these:

In [45]:
rows_before = df.shape[0]

# Filter event_ids with only one unique innings value
unique_innings = df.groupby('event_id')['innings'].nunique()
filtered_event_ids = unique_innings[unique_innings == 1].index

# Create a new dataframe without the filtered event_ids
df = df[~df['event_id'].isin(filtered_event_ids)]

df = df.reset_index(drop=True)

rows_after = df.shape[0]
print(rows_before-rows_after, 'rows removed')

809 rows removed


Finally, before we start seperately looking into build seperate dataframes for rating batters and bowlers, we can add preliminary columns for batter_rating and bowler_rating which we will come back to later:

In [ ]:
# Add preliminary columns for batter_rating and bowler_rating
df['batter_rating'] = 1
df['bowler_rating'] = 1

In [ ]:
def combine_pre_processing(df):

    if df['date'].dtype == 'object':
        # Convert dates to datetime
        df['date'] = df['date'].str[:10]
        df['date'] = pd.to_datetime(df['date'], format='mixed')

    # Identify event IDs with overs greater than 20
    event_ids_to_remove = df.loc[df['overs'] > 20, 'event_id'].unique()

    # Remove rows with event IDs that have overs greater than 20
    df = df.loc[~df['event_id'].isin(event_ids_to_remove)]

    # Identify event IDs with overs greater than 20
    event_ids_to_remove = df.loc[df['over_limit'] > 20, 'event_id'].unique()

    # Remove rows with event IDs that have overs greater than 20
    df = df.loc[~df['event_id'].isin(event_ids_to_remove)]

    # Create a dictionary of player id and name pairs
    role_id_names = {'bowler_id': 'bowler_name',
                'batter_id': 'batsman_striker_name',
                'nonstriker_id': 'batsman_nonstriker_name',
                'dismissal_bowler_id': 'dismissal_bowler_name',
                'dismissal_batsman_id': 'dismissal_batsman_name'}

    # Iterate through the dictionary and replace duplicates
    for id, name in role_id_names.items():
        replace_duplicate_names(df, id, name)
        print('\n')

    # Create a dictionary of team id and name pairs
    team_id_names = {'bowler_team_id': 'bowler_team_name',
                'batsman_striker_team_id': 'batsman_striker_team_name',
                'batsman_nonstriker_team_id': 'batsman_nonstriker_team_name'}

    # Iterate through the dictionary and replace duplicates
    for id, name in team_id_names.items():
        replace_duplicate_names(df, id, name)
        print('\n')

    # Drop rows where 'id' matches 999999999999999
    df = df.drop(df[df['id'] == 999999999999999].index)

    # Forward fill missing values based on 'event_id', 'innings', and the same value before the decimal point for 'overs'
    df['bowler_id'] = df.groupby(['event_id', 'innings', df['overs'].apply(int)])['bowler_id'].fillna(method='ffill')

    # Use the function to fill missing names from ID
    fill_name_from_id(df, 'bowler_id', 'bowler_name')

    # Fill missing values in 'bowler_id' and 'bowler_name' columns with 'Unknown'
    df['bowler_id'] = df['bowler_id'].fillna('Unknown')
    df['bowler_name'] = df['bowler_name'].fillna('Unknown')

    # Create a mask based on specific conditions
    mask = (
        ((df['score_value'].shift(1) % 2 == 0) & (df['wickets_lost'].shift(1) == 0)) &  # If the previous score is even and no wickets were lost
        (df['event_id'].shift(1) == df['event_id']) &  # If the previous event_id matches the current event_id
        (df['innings'].shift(1) == df['innings'])  # If the previous innings value matches the current innings value
    )

    # Initialise variables for missing count
    prev_missing_count = float('inf')
    curr_missing_count = df['batter_id'].isna().sum()

    # Iteratively fill missing values until the count stabilizes
    while curr_missing_count != prev_missing_count:
        prev_missing_count = curr_missing_count

        # Fill missing values in the 'batter_id' column using the previous non-missing value
        df.loc[mask, 'batter_id'] = df['batter_id'].fillna(df['batter_id'].shift(1))

        # Update the current missing count
        curr_missing_count = df['batter_id'].isna().sum()

    # Use the function to fill missing names from ID
    fill_name_from_id(df, 'batter_id', 'batsman_striker_name')

    # Fill missing values in 'batter_id' and 'batsman_striker_name' columnn with 'Unknown'
    df['batter_id'] = df['batter_id'].fillna('Unknown')
    df['batsman_striker_name'] = df['batsman_striker_name'].fillna('Unknown')

    # Set the initial value of the 'is_home_batter' column to 0 for all rows
    df['is_home_batter'] = 0

    # Update the 'is_home_batter' column to 1 for rows where the 'away_score' is '0'
    df.loc[df['away_score'] == '0', 'is_home_batter'] = 1

    df = replace_discrepancies_with_unknown(df, 'is_home_batter', 'batter_id', 'batsman_striker_name')

    # Assign 0 to the 'is_home_bowler' column for all rows in the DataFrame
    df['is_home_bowler'] = 0

    # Set the value of 'is_home_bowler' to 1 for rows where the 'home_score' column is '0'
    df.loc[df['home_score'] == '0', 'is_home_bowler'] = 1

    # Run the above function on discrepancies with is_home_bowler
    df = replace_discrepancies_with_unknown(df, 'is_home_bowler', 'bowler_id', 'bowler_name')

    # Step 2: Filter event_ids with only one unique innings value
    unique_innings = df.groupby('event_id')['innings'].nunique()
    filtered_event_ids = unique_innings[unique_innings == 1].index

    # Step 3: Create a new DataFrame without the filtered event_ids
    df = df[~df['event_id'].isin(filtered_event_ids)]

    # We can set preliminary batter and bowling ratings for future use 
    df['bowler_rating'] = 1
    df['batter_rating'] = 1

    # Finally we can reset the index
    df = df.reset_index(drop=True)

    return df


In [ ]:
clean_df = combine_pre_processing(df)

## Creating Features

We can first create functions to extract a dataset each for batting and bowling analysis:

In [ ]:
def get_bat_df(df):

    # Create a new dataframe bat_df
    bat_df = df[['event_id', 'innings', 'date', 'batter_id', 'batsman_striker_name', 'is_home_batter', 'batsman_striker_team_name', 'bowler_id', 'bowler_name', 'bowler_team_name', 'batter_balls_faced', 'batter_runs', 'home_score', 'away_score', 'dismissal_dismissal', 'batter_rating', 'bowler_rating']]

    return bat_df

In [ ]:
def get_bowl_df(df):

    # Create a new dataframe bowl_df
    bowl_df = df[['event_id', 'innings', 'date', 'bowler_id', 'bowler_name', 'bowler_team_name', 'batter_id', 'batsman_striker_name', 'batsman_striker_team_name', 'is_home_bowler', 'batter_balls_faced', 'batter_runs', 'home_score', 'away_score', 'dismissal_dismissal', 'batter_rating', 'bowler_rating', 'bowler_balls', 'bowler_conceded', 'bowler_wickets', 'innings_no_balls', 'innings_wides']]

    return bowl_df

We then create a function to create aggregated features for either the batting or bowling dataframe. (At the moment the batter_rating and bowler_ratings are all ones so finding the mean is redundant, but these will be updated later and this function will be run again)

In [ ]:
def create_agg_data(df, batter_or_bowler):

    if batter_or_bowler == 'batter':

        # Group the DataFrame 'bat_df' by 'batsman_striker_name' and 'event_id'
        grouped_data = df.groupby(['batsman_striker_name', 'event_id'])

        # Aggregate the grouped data using different aggregation functions for specific columns
        aggregated_batsman_data = grouped_data.agg({
            'batter_runs': 'last',
            'batter_balls_faced': 'last',
            'is_home_batter': 'mean',
            'dismissal_dismissal': 'last',
            'bowler_rating': 'mean'
        }).reset_index()

        return aggregated_batsman_data

    elif batter_or_bowler == 'bowler':

        # Group the DataFrame 'bowl_df' by 'bowler_name' and 'event_id'
        grouped_data = df.groupby(['bowler_name', 'event_id'])

        # Define a custom lambda function to count non-zero entries
        count_non_zero = lambda x: (x != 0).sum()

        # Aggregate the grouped data using different aggregation functions for specific columns
        aggregated_bowler_data = grouped_data.agg({
            'bowler_balls': 'max',
            'bowler_conceded': 'max',
            'bowler_wickets': 'max',
            'innings_wides': count_non_zero,
            'innings_no_balls': count_non_zero,
            'is_home_bowler': 'mean',
            'batter_rating': 'mean'
        }).reset_index()

        return aggregated_bowler_data

We can also create a function to create a dataframe containing aggregated data on each match including runs and wickets for each team:

In [ ]:
def get_agg_team_score_data(df):

    # Group the DataFrame 'df' by 'event_id' and 'innings'
    grouped_data = df.groupby(['event_id', 'bowler_team_id'])

    # Aggregate the grouped data using different aggregation functions for specific columns
    aggregated_innings_score_data = grouped_data.agg({
        'date': 'last',
        'home_score': 'last',
        'away_score': 'last',
        'match_ball_no': 'last'
    }).reset_index()

    # Group the 'aggregated_team_score_data' DataFrame by 'event_id'
    aggregated_team_score_data = aggregated_innings_score_data.groupby('event_id').agg({
        'date': 'last',
        'home_score': 'max',
        'away_score': 'max',
        'match_ball_no': ['last', 'first']
    }).reset_index()

    aggregated_team_score_data = pd.DataFrame(aggregated_team_score_data.values, columns=['event_id', 'date', 'home_score', 'away_score', 'home_balls', 'away_balls'])

    # Create new columns 'home_runs' and 'home_wickets' 
    aggregated_team_score_data['home_runs'] = aggregated_team_score_data['home_score'].str.split('/').str[0]
    aggregated_team_score_data['home_wickets'] = aggregated_team_score_data['away_score'].str.split('/').str[1]

    # Create new columns 'away_runs' and 'away_wickets' 
    aggregated_team_score_data['away_runs'] = aggregated_team_score_data['away_score'].str.split('/').str[0]
    aggregated_team_score_data['away_wickets'] = aggregated_team_score_data['home_score'].str.split('/').str[1]

    return aggregated_team_score_data

We next create a function to merge these two aggregated functions together:

In [ ]:
def merge_with_team_data(aggregated_team_score_data, aggregated_player_data):    
    
    # Merge 'aggregated_team_score_data' and 'aggregated_batsman_data' dataframes on 'event_id'
    df = pd.merge(aggregated_team_score_data, aggregated_player_data, on='event_id')

    return df

We can create a function to add a column stating whether the bowler or batter won their match:

In [ ]:
def create_player_won_columns(df, batter_or_bowler):

    # Create a new column named 'home_win'
    df['home_win'] = 0

    # Create a mask based on the condition of home_runs being greater than away_runs
    mask = (df['home_runs'] > df['away_runs'])

    # Assign the match result to the 'home_win' column using the mask
    df.loc[mask, 'home_win'] = 1

    if batter_or_bowler == 'batter':
        # Create a new column named 'batter_won' based on certain conditions
        df['batter_won'] = ((df['is_home_batter'] == 1) & (df['home_win'] == 1)) | ((df['is_home_batter'] == 0) & (df['home_win'] == 0))

    elif batter_or_bowler == 'bowler':
        # Create a new column named 'bowler_won' based on certain conditions
        df['bowler_won'] = ((df['is_home_bowler'] == 1) & (df['home_win'] == 1)) | ((df['is_home_bowler'] == 0) & (df['home_win'] == 0))

    return df
    

We can also add a column showing the games played for each player:

In [ ]:
def create_games_played_column(df, batter_or_bowler):

    if batter_or_bowler == 'batter':
        df['games_played'] = df.groupby('batsman_striker_name').cumcount() + 1

    elif batter_or_bowler == 'bowler':
        df['games_played'] = df.groupby('bowler_name').cumcount() + 1

    return df
        

We can now look into creating some additional features. 

One such feature we would be interetsed in would be the percentage of a certain metric (i.e. wickets/runs) a player scored of the total:

In [ ]:
def create_percentage_of_total_feature(df, player_data, home_data, away_data):

    # Convert columns to float type
    df[player_data] = df[player_data].astype(float)
    df[home_data] = df[home_data].astype(float)
    df[away_data] = df[away_data].astype(float) 

    # Define metrics
    metric = player_data.split('_')[1]
    percenatage_metric = '%_of_total_'+ metric

    # Calculate percentage
    df[percenatage_metric] = df[player_data]  / (df[home_data] + df[away_data])

    return df


Another important metric would be the number of runs a batter scored compared to how many balls he faced:

In [ ]:
def calculate_batter_run_rate(df):

    # Create a new column named 'run_rate'
    df['run_rate'] = np.where(
        df['batter_balls_faced'] == 0,  # If 'batter_balls_faced' is 0, set the run rate to 0
        0,
        df['batter_runs'] / df['batter_balls_faced']  # Calculate the run rate
    )

    return df

Similarly, for bowlers we would want a metric showing the opposition run rate:

In [ ]:
def create_run_rate_against(df):

    # Create a new column 'run_rate_against' 
    df['run_rate_against'] = np.where(
        # If 'bowler_balls' is 0, set the value to NaN
        df['bowler_balls'] == 0,
        np.nan,
        np.where(
            # If 'bowler_conceded' is 0, set the value to 0.5 divided by 'bowler_balls'
            df['bowler_conceded'] == 0,
            0.5 / df['bowler_balls'],
            # Otherwise, calculate the run rate by dividing 'bowler_conceded' by 'bowler_balls'
            df['bowler_conceded'] / df['bowler_balls']
        )
    )

    return df


Now we can create a function that adds all of these previously defined metrics onto our dataframes:

In [ ]:
def add_metrics_to_df(df, batter_or_bowler):

    if batter_or_bowler == 'batter':
        # Create %_of_total_runs column
        df = create_percentage_of_total_feature(df, 'batter_runs', 'home_runs', 'away_runs')
        
        # Create run_rate column
        df = calculate_batter_run_rate(df)

        # Create batter_won column
        df = create_player_won_columns(df, 'batter')

        # Create games_played column
        df = create_games_played_column(df, 'batter')
    
        # Extract necessary columns
        bat_df = df[['event_id', 'date', 'batsman_striker_name', 'batter_runs', 'games_played', '%_of_total_runs', 'run_rate', 'dismissal_dismissal', 'batter_won', 'bowler_rating']]

        return bat_df

    elif batter_or_bowler == 'bowler':
        # Create %_of_total_wickets
        df = create_percentage_of_total_feature(df, 'bowler_wickets', 'home_wickets', 'away_wickets')

        # Create %_of_total_balls
        df = create_percentage_of_total_feature(df, 'bowler_balls', 'home_balls', 'away_balls')

        # Create run_rate_against column
        df = create_run_rate_against(df)

        # Create batter_won column
        df = create_player_won_columns(df, 'bowler')

        # Create games_played column
        df = create_games_played_column(df, 'bowler')

        # Create new dataframe
        bowl_df = df[['event_id', 'date', 'bowler_name', 'bowler_wickets', 'games_played', '%_of_total_wickets', '%_of_total_balls', 'run_rate_against', 'bowler_won', 'batter_rating', 'innings_no_balls', 'innings_wides']]

        return bowl_df

We can then use these metrics to define scores for both batting and bowling while adding bonuses or penalties for certain instances such as the player's team winning the match, the batter not being dismissed or the bowler bowling wide or no-balls.

In [ ]:
def create_rating_score(df, batter_or_bowler):

    if batter_or_bowler == 'batter':

        # Define our feature rating_score
        df['rating_score'] = df['batter_runs'] * df['%_of_total_runs'] * df['run_rate']

        # Add 30% to the rating_score when batter_won is True
        df.loc[df['batter_won']==True, 'rating_score'] = df.loc[df['batter_won']==True, 'rating_score']* 1.3

        # Add 10% to the rating_score when the batter wasn't dismissed
        df.loc[df['dismissal_dismissal']==False, 'rating_score'] = df.loc[df['dismissal_dismissal']==False, 'rating_score'] * 1.1

        # Calculate the mean of the rating_score column for each player 
        df['player_mean'] = df.groupby('batsman_striker_name')['rating_score'].transform(lambda x: x.expanding().mean())

    if batter_or_bowler == 'bowler':

        # Define our wicket rating score
        df['wicket_rating_score'] = df['bowler_wickets'] * df['%_of_total_wickets'] 

        # Define our run rating score
        df['run_rating_score'] = df['%_of_total_balls'] / df['run_rate_against'] 
        df['run_rating_score'] = df['run_rating_score'].fillna(0)

        df['rating_score'] = (df['wicket_rating_score'] + df['run_rating_score']) / 2

        # Add 30% to the rating_score when batter_won is True
        df.loc[df['bowler_won']==True, 'rating_score'] = df.loc[df['bowler_won']==True, 'rating_score'] * 1.3

        # Take 95% of score for every foul or wide ball
        df['rating_score'] = df['rating_score'] * (0.95**(df['innings_no_balls']+df['innings_wides']))

        # Calculate the mean of the rating_score column for each player 
        df['player_mean'] = df.groupby('bowler_name')['rating_score'].transform(lambda x: x.expanding().mean())

    return df

We can create a function that will normalise our scores around a mean of one and add upper and lower bounds if necessary:

In [ ]:
def normalise_scores(df, score, lower_bound, upper_bound):

    # Calculate the mean of the score column up to and including the current row
    df['mean'] = df[score].expanding().mean()

    # Calculate the scaling factor to normalize the distribution up to and including the current row
    df['scale_factor'] = 1 / df['mean']

    # Define column name
    score_normalised = score + '_normalised'

    # Apply the scaling factor to the rating_score column up to and including the current row
    df[score_normalised] = df[score] * df['scale_factor']

    # Apply upper and lower bound
    df[score_normalised] = np.clip(df[score_normalised], lower_bound, upper_bound)

    # Drop unnecessary columns
    df = df.drop(['mean', 'scale_factor'], axis=1)

    return df

We can then create a function that adds our scores and normalised scores to our dataframe:

In [ ]:
def add_rating_features(df, batter_or_bowler):

    if batter_or_bowler == 'batter':
    
        # Add batter_rating and player_mean
        df = create_rating_score(df, 'batter')
        
        # Normalise rating_score
        df = normalise_scores(df, 'rating_score', 0, 10)

        # Normalise player_mean
        df = normalise_scores(df, 'player_mean', 0.7, 1.3)
        # Set as 1 for the first 10 games played
        df.loc[df['games_played'] <= 10, 'player_mean_normalised'] = 1

        # Normalise bowler_rating
        df = normalise_scores(df, 'bowler_rating', 0.5, 1.5)

    if batter_or_bowler == 'bowler':
        
        # Add bowler_rating and player_mean
        df = create_rating_score(df, 'bowler')

        # Normalise rating_score
        df = normalise_scores(df, 'rating_score', 0, 10)

        # Normalise player_mean
        df = normalise_scores(df, 'player_mean', 0.7, 1.3)
        # Set as 1 for the first 10 games played
        df.loc[df['games_played'] <= 10, 'player_mean_normalised'] = 1

        # Normalise batter_rating
        df = normalise_scores(df, 'batter_rating', 0.5, 1.5)

    return df


We can create a function that combines all of the above steps:

In [ ]:
def pre_processing(df, batter_or_bowler):
    
    if batter_or_bowler == 'batter':
    
        bat_df = create_agg_data(df, 'batter')
    
        bat_df = merge_with_team_data(aggregated_team_score_data, bat_df)   

        bat_df = add_metrics_to_df(bat_df, 'batter')
        
        bat_df = create_rating_score(bat_df, 'batter')

        bat_df = add_rating_features(bat_df, 'batter')

        return bat_df

    elif batter_or_bowler == 'bowler':

        bowl_df = create_agg_data(df, 'bowler')

        bowl_df = merge_with_team_data(aggregated_team_score_data, bowl_df)   

        bowl_df = add_metrics_to_df(bowl_df, 'bowler')

        bowl_df = create_rating_score(bowl_df, 'bowler')

        bowl_df = add_rating_features(bowl_df, 'bowler')

        return bowl_df

Now that we have created both our batting and bowling dataframes, we can now create a function that adds batting and bowling rating scores back to our orginal cleaned dataframe. We can then re-run the previous function so that we now have a mean score rating for each opposing player per match: 

In [ ]:
def update_rating_score(original_df, bat_df, bowl_df): 

    # Create a bowler_rating score
    bowl_df['bowler_rating_score'] = bowl_df['rating_score_normalised'] * np.log1p(bowl_df['games_played'])

    # Merge witrh original df
    original_df = pd.merge(original_df, bowl_df[['event_id', 'bowler_name', 'bowler_rating_score']], on=['event_id', 'bowler_name'], how='left')
    original_df['bowler_rating'] = original_df['bowler_rating_score']


    # Create a batter_rating score
    bat_df['batter_rating_score'] = bat_df['rating_score_normalised'] * np.log1p(bat_df['games_played'])

    # Merge witrh original df
    original_df = pd.merge(original_df, bat_df[['event_id', 'batsman_striker_name', 'batter_rating_score']], on=['event_id', 'batsman_striker_name'], how='left')
    original_df['batter_rating'] = original_df['batter_rating_score']

    # Get bat_df from the orginal df with the updated bowler_rating 
    bat_df2 = get_bat_df(original_df)

    # Get bowl_df from the orginal df with the updated batter_rating
    bowl_df2 = get_bowl_df(original_df)

    # Run through the pre-processing steps again - batting
    bat_df2 = pre_processing(bat_df2, 'batter')

    # Run through the pre-processing steps again - bowling
    bowl_df2 = pre_processing(bowl_df2, 'bowler')

    # Drop new columns from original df
    original_df.drop(['bowler_rating_score', 'batter_rating_score'], axis=1, inplace=True)

    return bat_df2 , bowl_df2   

Finally we can combine the prepocessing steps with the above function that updates the player scores and re-runs the process to get our final dataframes for our algorithm: 

In [ ]:
def create_bat_bowl_dfs(clean_df):
    # Obtain the batting DataFrame
    bat_df = get_bat_df(clean_df)
    
    # Obtain the bowling DataFrame
    bowl_df = get_bowl_df(clean_df)
    
    # Get aggregated team score data (make it global so it can be used in exterior functions)
    global aggregated_team_score_data
    aggregated_team_score_data = get_agg_team_score_data(clean_df)
    
    # Perform pre-processing on the batting DataFrame
    bat_df2 = pre_processing(bat_df, 'batter')
    
    # Perform pre-processing on the bowling DataFrame
    bowl_df2 = pre_processing(bowl_df, 'bowler')
    
    # Update the rating score based on dataframes
    bat_df3, bowl_df3 = update_rating_score(clean_df, bat_df2, bowl_df2)
    
    return bat_df3, bowl_df3


We can define our batting and bowling dataframes using the above function:

In [ ]:
bat_df, bowl_df = create_bat_bowl_dfs(clean_df)

## Rating Algorithm

First can define a function that will normalise our final rating scores on a scale out of 1000 (adding 10% to the max value to allow for improvement):

In [ ]:
def normalise_rating_scale(df, rating):

    min_value = df[rating].min()
    max_value = df[rating].max() * 1.1

    df[rating] = (df[rating] - min_value) * (1000 / (max_value - min_value))

    return df

Finally we can build our algorithm that takes the weighted average that sums the product normalised rating score, the normlaised opponents rating, the normalised players performance compared to their mean and the log of the number of games played, and divided it by the cumululative sum of the log of games played (with a power of 0.75 to add extra weight to players who have played more games)

In [ ]:
def create_overall_rating_df(bat_df, bowl_df):

    # Batters

    # Create a new column for the overall_rating_batting
    df['overall_rating_batting'] = 0.0

    # Iterate over each unique batsman_striker_name
    for batsman in bat_df['batsman_striker_name'].unique():
        # Get the subset of rows for the current batsman
        subset = bat_df[bat_df['batsman_striker_name'] == batsman]

        # Initialise the cumulative sum variable
        cumulative_sum = 0.0

        # Iterate over each row in the subset
        for i in range(len(subset)):
            # Update the cumulative sum of the log of the games played
            cumulative_sum += np.log1p(subset.iloc[i]['games_played'])

            # Calculate the overall rating
            overall_rating = (subset.iloc[:i+1]['rating_score_normalised'] * # Normalised rating score (capped at 10)
                                subset.iloc[:i+1]['bowler_rating_normalised'] * # Normalised opponent rating (capped between 0.5 and 1.5)
                                np.log1p(subset.iloc[:i+1]['games_played']) * # Log of games played
                                subset.iloc[:i+1]['player_mean_normalised']).sum() / cumulative_sum ** 0.75 # We take the cumulative sum to the power of 0.75 to favour players who have played longer

            # Assign the weighted average to the corresponding row
            bat_df.at[subset.index[i], 'overall_rating_batting'] = overall_rating

    # Normalise rating scale
    bat_df = normalise_rating_scale(bat_df, 'overall_rating_batting')

    # Bowlers

    # Create a new column for the overall_rating_bowling
    df['overall_rating_bowling'] = 0.0

    # Iterate over each unique bowler_name
    for bowler in bowl_df['bowler_name'].unique():
        # Get the subset of rows for the current batsman
        subset = bowl_df[bowl_df['bowler_name'] == bowler]

        # Initialize the cumulative sum variable
        cumulative_sum = 0.0

        # Iterate over each row in the subset
        for i in range(len(subset)):
            # Update the cumulative sum of the log of the games played
            cumulative_sum += np.log1p(subset.iloc[i]['games_played'])

            # Calculate the overall rating
            overall_rating = (subset.iloc[:i+1]['rating_score_normalised'] * # Normalised rating score (capped at 10)
                                subset.iloc[:i+1]['batter_rating_normalised'] * # Normalised opponent rating (capped between 0.5 and 1.5)
                                np.log1p(subset.iloc[:i+1]['games_played'])  * # Log of games played
                                subset.iloc[:i+1]['player_mean_normalised'] ).sum() / cumulative_sum ** 0.75 # We take the cumulative sum to the power of 0.75 to favour players who have played longer

            # Assign the weighted average to the corresponding row
            bowl_df.at[subset.index[i], 'overall_rating_bowling'] = overall_rating

    # Normalise rating scale
    bowl_df = normalise_rating_scale(bowl_df, 'overall_rating_bowling')

    return bat_df, bowl_df


                
                    

We can create the following function to run the above function and combine the two dataframes while also creating a rating value for all rounders:

In [ ]:
def get_full_ratings_df(bat_df, bowl_df):

    bat_df2, bowl_df2 = create_overall_rating_df(bat_df, bowl_df)

    # We rename our player names to be consistent
    bat_df2 = bat_df2.rename({'batsman_striker_name': 'player_name'}, axis=1)
    bowl_df2 = bowl_df2.rename({'bowler_name': 'player_name'}, axis=1)

    # We merge our dataframes
    merge_df = pd.merge(bat_df2, bowl_df2, on=['player_name', 'event_id', 'date'], suffixes=('_bat', '_bowl'), how='outer')

    # Fill any nan values 
    merge_df = merge_df.fillna(0)

    # Create an all-rounder column
    merge_df['overall_rating_all_rounder'] = (merge_df['overall_rating_batting'] * merge_df['overall_rating_bowling']) / 1000

    # Create games played for all-rounders
    merge_df['games_played_all_round'] = merge_df.groupby('player_name').cumcount() + 1

    return merge_df

We can run the function to get our rating dataframe:

In [ ]:
merge_df = get_full_ratings_df(bat_df, bowl_df)

We can create a function to get mean monthly ratings and a player's ranking for each month:

In [ ]:
import pandas as pd

def get_rankings(df, rating_type):
    rating_column = 'overall_rating_' + rating_type

    # Get month and year
    df['quarter'] = df['date'].dt.quarter
    df['year'] = df['date'].dt.year

    # Create a dataframe with all possible combinations of 'player_name', 'month', and 'year'
    all_combinations = pd.MultiIndex.from_product([df['player_name'].unique(), df['quarter'].unique(), df['year'].unique()], 
                                                  names=['player_name', 'quarter', 'year'])
    all_combinations = all_combinations.to_frame(index=False)

    # Merge the original data with all combinations to fill missing values
    filled_data = pd.merge(all_combinations, df, how='left', on=['player_name', 'quarter', 'year'])

    # Forward fill missing values
    filled_data[rating_column].fillna(method='ffill', inplace=True)

    # Group the data by 'player_name', 'month', and 'year', and find the mean rating for each group
    grouped_data = filled_data.groupby(['player_name', 'quarter', 'year']).agg({rating_column: 'mean'}).reset_index()

    # Sort the data by 'year', 'month', and rating in descending, ascending, and descending order respectively
    sorted_data = grouped_data.sort_values(['year', 'quarter', rating_column], ascending=[False, False, False])

    # Reset the index of the sorted data and rename the columns
    sorted_data.reset_index(drop=True, inplace=True)
    sorted_data.columns = ['player_name', 'quarter', 'year', rating_column]

    # Assign the ranking based on the sorted data for each month
    sorted_data['ranking'] = sorted_data.groupby(['year', 'quarter'])[rating_column].rank(method='dense', ascending=False)

    return sorted_data

We can create the ranking dataframes and save them as files:

In [ ]:
def merge_create_and_save_files(df):

    # Create our ranking dataframes
    df_batting = get_rankings(df, 'batting')
    df_bowling = get_rankings(df, 'bowling')
    df_all_rounder = get_rankings(df, 'all_rounder')

    # Create  year and quarter columns on our dataframe
    df['quarter'] = pd.to_datetime(df['date']).dt.quarter
    df['year'] = pd.to_datetime(df['date']).dt.year

    # Find the max of each games played at each quarter
    max_games_played_bat = df.groupby(['player_name', 'year', 'quarter'])['games_played_bat'].max().reset_index()
    max_games_played_bowl = df.groupby(['player_name', 'year', 'quarter'])['games_played_bowl'].max().reset_index()
    max_games_played_all_round = df.groupby(['player_name', 'year', 'quarter'])['games_played_all_round'].max().reset_index()

    # Merge dataframes to get games played at each quarter
    rank_df_batting = pd.merge(df_batting, max_games_played_bat[['player_name', 'year', 'quarter', 'games_played_bat']], on=['player_name', 'year', 'quarter'], how='left') 
    rank_df_bowling = pd.merge(df_bowling, max_games_played_bowl[['player_name', 'year', 'quarter', 'games_played_bowl']], on=['player_name', 'year', 'quarter'], how='left') 
    rank_df_all_rounder = pd.merge(df_all_rounder, max_games_played_all_round[['player_name', 'year', 'quarter', 'games_played_all_round']], on=['player_name', 'year', 'quarter'], how='left') 

    # Back fill any missing values and fill the rest with 0
    rank_df_batting['games_played_bat'] = rank_df_batting.groupby('player_name')['games_played_bat'].fillna(method='bfill').fillna(1)
    rank_df_bowling['games_played_bowl'] = rank_df_bowling.groupby('player_name')['games_played_bowl'].fillna(method='bfill').fillna(1)
    rank_df_all_rounder['games_played_all_round'] = rank_df_all_rounder.groupby('player_name')['games_played_all_round'].fillna(method='bfill').fillna(1)

    # Save to files
    rank_df_batting.to_csv(r'datasets\batting_rankings.csv', index=False)
    rank_df_bowling.to_csv(r'datasets\bowling_rankings.csv', index=False)
    rank_df_all_rounder.to_csv(r'datasets\all_rounder_rankings.csv', index=False)

    return rank_df_batting, rank_df_bowling, rank_df_all_rounder

In [ ]:
rank_df_batting, rank_df_bowling, rank_df_all_rounder = merge_create_and_save_files(merge_df)

We can finally create a function to extract a player's rating and ranking for a certain month and year:

In [ ]:
def get_player_rating_ranking(df, player_name, rating_type, year, quarter):

    rank_df = get_rankings(df, rating_type)

    row = rank_df[(rank_df['player_name'] == player_name) & (rank_df['year']==year) & (rank_df['quarter']==quarter)]

    rating = row.at[row.index[0], 'overall_rating_'+rating_type].round(2)
    ranking = int(row.at[row.index[0], 'ranking'])

    result = {'Player Name': player_name, 'Rating': rating, 'Ranking': ranking}

    return result 
